In [6]:
import numpy as np

def metodo_gauss_seidel(w, sigma, u, e, alpha_inicial=1.0, beta_inicial=1.0, max_iter=1000, tol=1e-6):
    u_0 = 4 * np.pi * 1e-7
    e_0 = (1e-9) / (36 * np.pi)
    
    # Verificar si es un buen conductor
    if sigma / (w * e) > 10:  # sigma / (w * e) >> 1
        alpha = beta = np.sqrt((u * w * sigma)/(2))
        return alpha, beta
    
    else:
        alpha = alpha_inicial
        beta = beta_inicial
        for i in range(max_iter):
            # Guardar valores antiguos para verificar la convergencia
            alpha_antiguo = alpha
            beta_antiguo = beta

            # Actualizar alpha usando el valor actual de beta
            alpha = w * sigma * u / (2 * beta)
            
            # Actualizar beta usando el nuevo valor de alpha
            beta = (alpha**2 + w**2 * u * e)**0.5

            # Verificar la convergencia
            if abs(alpha - alpha_antiguo) < tol and abs(beta - beta_antiguo) < tol:
                break

        return alpha, beta

# Prueba
w = 1e8
sigma = 3
u = 20*4*np.pi*1e-7
e = (1e-9)/(36*np.pi)

alpha, beta = metodo_gauss_seidel(w, sigma, u, e)
print(f"Alpha: {alpha}, Beta: {beta}")

Alpha: 61.399602476789305, Beta: 61.399602476789305
